In [1]:
!pip install opendatasets datasets accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs?select=NvidiaDocumentationQandApairs.csv")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: halukcelebi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs


100%|██████████| 400k/400k [00:00<00:00, 807kB/s]

In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import re

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [6]:
data = pd.read_csv("/content/nvidia-documentation-question-and-answer-pairs/NvidiaDocumentationQandApairs.csv")[["question", "answer"]]

In [7]:
data.head()

,question,answer
0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...


In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub('[^A-Za-z0-9]+', ' ',text)
    return text

In [9]:
text = "Who was Mike $*?"
clean_text(text)

'who was mike '

In [10]:
data['question'] = data['question'].apply(clean_text)

In [11]:
data['answer'] = data['answer'].apply(clean_text)

In [12]:
data.head()

,question,answer
0,what is hybridizer,hybridizer is a compiler from altimesh that en...
1,how does hybridizer generate optimized code,hybridizer uses decorated symbols to express p...
2,what are some parallelization patterns mention...,the text mentions using parallelization patter...
3,how can you benefit from accelerators without ...,you can benefit from accelerators compute hors...
4,what is an example of using hybridizer,an example in the text demonstrates using para...


In [13]:
train = data.sample(frac = 0.7)
test = data.drop(train.index)
val = test.sample(frac= 0.5)
test = test.drop(val.index)


In [14]:
print("Training Shape", train.shape)
print("Validation Shape",val.shape )
print("Testing Shape", test.shape)

Training Shape (4976, 2)
Validation Shape (1066, 2)
Testing Shape (1066, 2)


In [15]:
original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype = torch.float32)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [17]:
def tokenize_function(example):
  start_prompt = "According to the following question:\n\n"
  end_prompt = "\nAnswer:\n\n"

  prompt = [start_prompt + question + end_prompt for question in example["question"] ]
  example["input_ids"] = tokenizer(prompt, padding = 'max_length', truncation= True, return_tensors = 'pt').input_ids
  example["labels"] = tokenizer(example["answer"], padding = 'max_length', truncation = True, return_tensors = 'pt').input_ids

  return example


In [18]:
train_data = Dataset.from_pandas(train)
train_tokenized_datasets = train_data.map( tokenize_function, batched = True )
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['question', 'answer','__index_level_0__'])


Map:   0%|          | 0/4976 [00:00<?, ? examples/s]

In [19]:
train_tokenized_datasets

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4976
})

In [20]:
val_data = Dataset.from_pandas(val)
val_tokenized_datasets = val_data.map( tokenize_function, batched = True )
val_tokenized_datasets = val_tokenized_datasets.remove_columns(['question', 'answer','__index_level_0__'])

test_data = Dataset.from_pandas(test)
test_tokenized_datasets = test_data.map( tokenize_function, batched = True )
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['question', 'answer','__index_level_0__'])


Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [21]:
EPOCHS = 5
LR = 1e-3
BATCH_SIZE = 2

training_path = "./training_nvidia_chatbot"
training_args = TrainingArguments(
    output_dir = training_path,
    save_total_limit = 2,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LR,
    num_train_epochs = EPOCHS,
    evaluation_strategy = 'epoch'
)

trainer = Trainer(
    model = original_model,
    args = training_args,
    train_dataset = train_tokenized_datasets,
    eval_dataset = val_tokenized_datasets
)

trainer.train()
model_path = "./nvidia_chatbot_final_model"

trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.181900,0.168932
2,0.137600,0.150752
3,0.097300,0.144496
4,0.060900,0.154853
5,0.037800,0.170736


('./nvidia_chatbot_final_model/tokenizer_config.json',
 './nvidia_chatbot_final_model/special_tokens_map.json',
 './nvidia_chatbot_final_model/spiece.model',
 './nvidia_chatbot_final_model/added_tokens.json',
 './nvidia_chatbot_final_model/tokenizer.json')

In [22]:
eval_results = trainer.evaluate(eval_dataset=test_tokenized_datasets)

In [23]:
test_data[53]

{'question': 'how did rapids cudf integrate gpu hash maps and what benefits did it offer ',
 'answer': 'rapids cudf integrated gpu hash maps resulting in significant speedups for data science workloads this integration leveraged the power of gpus to enhance data processing efficiency ',
 '__index_level_0__': 431}

In [24]:
test_text = "what is AI"
start_prompt = "According to the following question:\n\n"
end_prompt = "\nAnswer:\n\n"
full_prompt = start_prompt + test_text + end_prompt
print(full_prompt)

According to the following question:

what is AI
Answer:




In [25]:
from transformers import GenerationConfig

trained_model =  AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenized_test_text = tokenizer(full_prompt,return_tensors = 'pt').input_ids.to(device)

model_output = trained_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150)
                                      )[0]

final_output = tokenizer.decode(model_output,skip_special_tokens=True)
print(final_output)

AI is a machine learning algorithm used to understand complex systems and algorithms it is designed to help developers create predictive models that improve their accuracy 


In [26]:
from transformers import GenerationConfig

original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype = torch.float32).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

tokenized_test_text = tokenizer(full_prompt,return_tensors = 'pt').input_ids.to(device)

model_output = original_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150)
                                      )[0]

final_output = tokenizer.decode(model_output,skip_special_tokens=True)
print(final_output)

a computer program
